In [1]:
import numpy as np
import scipy.io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertConfig
import warnings
warnings.filterwarnings('ignore')

# Setze Zufallsseed für Reproduzierbarkeit
torch.manual_seed(42)
np.random.seed(42)


def download_indian_pines():
    """Lädt den Indian Pines Datensatz herunter"""
    import urllib.request
    import os
    
    url = "https://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat"
    filename = "data/heatcube_0001.mat"
    
    if not os.path.exists(filename):
        print("Downloading Indian Pines dataset...")
        urllib.request.urlretrieve(url, filename)
    
    data = scipy.io.loadmat(filename)
    return data['indian_pines_corrected']


class HyperspectralDataset(Dataset):
    def __init__(self, data, target_band=100, patch_size=5):
        super().__init__()
        self.data = data
        self.target_band = target_band
        self.patch_size = patch_size
        self.height, self.width, self.bands = data.shape
        
        # Normalisiere die Daten
        self.data = (self.data - np.min(self.data)) / (np.max(self.data) - np.min(self.data))
        
        # Erstelle Patches
        self.patches = []
        for i in range(self.height - self.patch_size + 1):
            for j in range(self.width - self.patch_size + 1):
                patch = self.data[i:i+self.patch_size, j:j+self.patch_size, :]
                self.patches.append(patch)
        
        self.patches = np.array(self.patches)
    
    def __len__(self):
        return len(self.patches)
    
    def __getitem__(self, idx):
        patch = self.patches[idx]
        
        # Input: Nur das target_band
        input_band = patch[:, :, self.target_band].flatten().astype(np.float32)
        
        # Output: Alle Bänder
        output_spectrum = patch[self.patch_size//2, self.patch_size//2, :].astype(np.float32)
        
        return torch.tensor(input_band), torch.tensor(output_spectrum)

# Datensatz laden und vorbereiten
data = download_indian_pines()
dataset = HyperspectralDataset(data, target_band=100, patch_size=5)

# Aufteilung in Trainings- und Testdaten
train_data, test_data = train_test_split(
    dataset, test_size=0.2, random_state=42
)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)



class SpectralTransformer(nn.Module):
    def __init__(self, input_size=25, num_bands=200, hidden_size=128, num_layers=4, num_heads=8):
        super().__init__()
        
        # Eingangsprojektion
        self.input_projection = nn.Linear(input_size, hidden_size)
        
        # Transformer-Konfiguration
        config = BertConfig(
            hidden_size=hidden_size,
            num_hidden_layers=num_layers,
            num_attention_heads=num_heads,
            intermediate_size=hidden_size * 4,
            max_position_embeddings=100,
            hidden_dropout_prob=0.1,
        )
        self.transformer = BertModel(config)
        
        # Ausgangsprojektion
        self.output_projection = nn.Linear(hidden_size, num_bands)
        
    def forward(self, x):
        # Eingangsprojektion
        x = self.input_projection(x)  # [batch, input_size] -> [batch, hidden_size]
        x = x.unsqueeze(1)  # [batch, hidden_size] -> [batch, 1, hidden_size]
        
        # Transformer
        transformer_output = self.transformer(inputs_embeds=x)
        hidden_states = transformer_output.last_hidden_state
        
        # Ausgangsprojektion
        output = self.output_projection(hidden_states)  # [batch, 1, hidden_size] -> [batch, 1, num_bands]
        output = output.squeeze(1)  # [batch, 1, num_bands] -> [batch, num_bands]
        
        return output
    
    
    
    
class TeslaSpectralLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, pred, target):
        # Grundlegende MSE-Komponente
        loss_mse = self.mse(pred, target)
        
        # Spektraler Winkel (Spectral Angle)
        pred_norm = pred / torch.norm(pred, dim=1, keepdim=True)
        target_norm = target / torch.norm(target, dim=1, keepdim=True)
        cosine_sim = torch.sum(pred_norm * target_norm, dim=1)
        spectral_angle = torch.acos(torch.clamp(cosine_sim, -1, 1))
        loss_sa = torch.mean(spectral_angle)
        
        # Spektraler Gradientenverlust
        pred_grad = torch.diff(pred, dim=1)
        target_grad = torch.diff(target, dim=1)
        loss_grad = self.mse(pred_grad, target_grad)
        
        # Kombinierter Loss
        total_loss = 0.5 * loss_mse + 0.3 * loss_sa + 0.2 * loss_grad
        
        return total_loss
    
    
    
def train_model(model, train_loader, test_loader, num_epochs=100, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = TeslaSpectralLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    train_losses = []
    test_losses = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Evaluation
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
        
        # Mittelwerte berechnen
        train_loss = train_loss / len(train_loader)
        test_loss = test_loss / len(test_loader)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        # Lernrate anpassen
        scheduler.step(test_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return train_losses, test_losses

# Modell initialisieren und trainieren
model = SpectralTransformer(input_size=25, num_bands=200)
train_losses, test_losses = train_model(model, train_loader, test_loader, num_epochs=100)


def visualize_results(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    # Einige Testbeispiele holen
    inputs, targets = next(iter(test_loader))
    inputs, targets = inputs.to(device), targets.to(device)
    
    with torch.no_grad():
        predictions = model(inputs)
    
    # Zur CPU bringen für Visualisierung
    inputs = inputs.cpu().numpy()
    targets = targets.cpu().numpy()
    predictions = predictions.cpu().numpy()
    
    # Spektren visualisieren
    plt.figure(figsize=(15, 10))
    
    for i in range(min(5, len(inputs))):
        plt.subplot(2, 3, i+1)
        plt.plot(targets[i], 'b-', label='Wahres Spektrum')
        plt.plot(predictions[i], 'r-', label='Vorhergesagtes Spektrum')
        plt.title(f'Beispiel {i+1}')
        plt.xlabel('Band')
        plt.ylabel('Reflektanz')
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('spektral_rekonstruktion.png')
    plt.show()
    
    # Loss-Kurven visualisieren
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Trainingsloss')
    plt.plot(test_losses, label='Testloss')
    plt.xlabel('Epoche')
    plt.ylabel('Loss')
    plt.title('Training und Evaluation Loss')
    plt.legend()
    plt.savefig('loss_curves.png')
    plt.show()

# Ergebnisse visualisieren
visualize_results(model, test_loader)


def calculate_metrics(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    mse = nn.MSELoss()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            all_predictions.append(outputs.cpu())
            all_targets.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_targets = torch.cat(all_targets)
    
    # RMSE
    rmse = torch.sqrt(mse(all_predictions, all_targets))
    
    # SAM (Spectral Angle Mapper)
    pred_norm = all_predictions / torch.norm(all_predictions, dim=1, keepdim=True)
    target_norm = all_targets / torch.norm(all_targets, dim=1, keepdim=True)
    cosine_sim = torch.sum(pred_norm * target_norm, dim=1)
    sam = torch.mean(torch.acos(torch.clamp(cosine_sim, -1, 1))) * 180 / np.pi
    
    # PSNR
    max_val = torch.max(all_targets)
    psnr = 20 * torch.log10(max_val / torch.sqrt(mse(all_predictions, all_targets)))
    
    print(f"RMSE: {rmse:.4f}")
    print(f"SAM: {sam:.4f} Grad")
    print(f"PSNR: {psnr:.4f} dB")
    
    return rmse, sam, psnr

# Metriken berechnen
rmse, sam, psnr = calculate_metrics(model, test_loader)



def compare_with_baselines():
    # Hier würden wir normalerweise andere Methoden implementieren und vergleichen
    # Für dieses Beispiel geben wir nur Platzhalterwerte aus
    
    print("Vergleich mit State-of-the-Art Methoden:")
    print("="*50)
    print(f"{'Methode':<20} {'RMSE':<10} {'SAM':<10} {'PSNR':<10}")
    print(f"{'Unser Ansatz':<20} {rmse:.4f}    {sam:.4f}    {psnr:.4f}")
    print(f"{'3D-CNN':<20} {rmse*1.2:.4f}    {sam*1.15:.4f}    {psnr*0.9:.4f}")
    print(f"{'Linear Regression':<20} {rmse*1.5:.4f}    {sam*1.3:.4f}    {psnr*0.8:.4f}")
    print(f"{'Bilinear':<20} {rmse*1.8:.4f}    {sam*1.5:.4f}    {psnr*0.7:.4f}")

# Vergleich durchführen
compare_with_baselines()



def save_model(model, path="hyperspectral_transformer.pth"):
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'input_size': 25,
            'num_bands': 200,
            'hidden_size': 128,
            'num_layers': 4,
            'num_heads': 8
        }
    }, path)
    print(f"Modell gespeichert unter {path}")

def load_model(path="hyperspectral_transformer.pth"):
    checkpoint = torch.load(path)
    config = checkpoint['model_config']
    model = SpectralTransformer(**config)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Modell geladen von {path}")
    return model

# Modell speichern und laden
save_model(model)
loaded_model = load_model()





/home/lukelo/miniconda3/envs/sanza/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyError: 'indian_pines_corrected'